In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.cluster.util import cosine_distance
import numpy as np

def summarize_text(text, num_sentences):
    sentences = sent_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()

    # pre-process the sentences
    words_list = []
    for sentence in sentences:
        words = tokenizer.tokenize(sentence)
        words = [ps.stem(word.lower()) for word in words if word.lower() not in stop_words]
        words_list.append(words)

    # calculate the sentence similarity matrix
    sim_matrix = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_matrix[i][j] = cosine_distance(words_list[i], words_list[j])

    # apply page rank algorithm to get sentence scores
    sentence_scores = np.array([sum(sim_matrix[i]) for i in range(len(sim_matrix))])
    ranked_sentences = [sentences[i] for i in sentence_scores.argsort()[-num_sentences:]]

    return ' '.join(ranked_sentences)
